<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_time_series/class_01/TS%20-%20W1%20-%2011%20-%20Atividade_Avaliativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Laboratório Prático: Trabalhando com Séries Temporais

Bem-vindo! Nesta tarefa, você trabalhará com dados de séries temporais.

Todos os dados serão gerados(sintéticos) e você implementará várias funções para
* dividir os dados,
* criar previsões e
* avaliar a qualidade dessas previsões.


Vamos começar!

In [ ]:
# Baixar arquivos adicionais para o laboratório
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/ml_intro/class_02/5%20-%20Atividade%20Avaliativa%20-%20Regress%C3%A3o%20Linear/lab_utils_ml_intro_assig_week_2.zip
!unzip -n -q lab_utils_ml_intro_assig_week_2.zip

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

A próxima célula inclui várias funções auxiliares para gerar e plotar a série temporal:

In [ ]:
def trend(time, slope=0):
    """Uma tendência ao longo do tempo"""
    return slope * time

def seasonal_pattern(season_time):
    """Apenas um padrão arbitrário"""
    return np.where(season_time < 0.1,
                    np.cos(season_time * 7 * np.pi),
                    1 / np.exp(5 * season_time))


def seasonality(time, period, amplitude=1, phase=0):
    """Repete o mesmo padrão em cada período"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)


def noise(time, noise_level=1, seed=None):
    """Adiciona ruído à série"""
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level


def plot_series(time, series, format="-", title="", label=None, start=0, end=None):
    """Plotar a série"""
    plt.plot(time[start:end], series[start:end], format, label=label)
    plt.xlabel("Tempo")
    plt.ylabel("Valor")
    plt.title(title)
    if label:
        plt.legend()
    plt.grid(True)

## Gerar dados de série temporal

Usando as funções anteriores, gere dados que se assemelhem a uma série temporal da vida real.

Observe que `TIME` representa os valores na coordenada x, enquanto `SERIES` representa os valores na coordenada y. Essa nomenclatura é usada para evitar confusão com outros tipos de dados nos quais `x` e `y` têm significados diferentes.

In [ ]:
# A dimensão de tempo ou a coordenada x da série temporal
TIME = np.arange(4 * 365 + 1, dtype="float32")

# A série inicial é apenas uma linha reta com uma interceptação y
y_intercept = 10
slope = 0.01
SERIES = trend(TIME, slope) + y_intercept

# Adição de sazonalidade
amplitude = 40
SERIES += seasonality(TIME, period=365, amplitude=amplitude)

# Adicionando algum ruído
noise_level = 2
SERIES += noise(TIME, noise_level, seed=42)

# Plotar a série
plt.figure(figsize=(10, 6))
plot_series(TIME, SERIES)
plt.show()

Agora que temos a série temporal, vamos dividi-la para que possamos iniciar a previsão.

Preencha a função `train_val_split` abaixo, que recebe os dados `time` (coordenada x) e `series` (coordenada y) juntamente com o `time_step` no qual a divisão deve ser realizada. Observe que o padrão desse valor é 1100, pois essa é uma etapa apropriada para dividir a série em treinamento e validação:

In [ ]:
# Definir a etapa de tempo para dividir a série
SPLIT_TIME = 1100

def train_val_split(time, series, time_step=SPLIT_TIME):

    ### INICIE SEU CÓDIGO AQUI
    time_train = None
    series_train = None
    time_valid = None
    series_valid = None
    ### TERMINE SEU CÓDIGO AQUI

    return time_train, series_train, time_valid, series_valid

In [ ]:
# Teste sua função
time_train, series_train, time_valid, series_valid = train_val_split(TIME, SERIES)

plt.figure(figsize=(10, 6))
plot_series(time_train, series_train, title="Treino")
plt.show()

plt.figure(figsize=(10, 6))
plot_series(time_valid, series_valid, title="Validação")
plt.show()

**Saída Esperada:**

<table><tr><td><img src='images/train_series.png'></td><td><img src='images/val_series.png'></td></tr></table>

## Métricas de avaliação

Agora que você dividiu com sucesso os dados em conjuntos de treinamento e validação, precisará de uma maneira de saber se suas previsões são boas. Para isso, execute a função `compute_metrics` abaixo. Essa função recebe a série verdadeira e a previsão e retorna o `mse` e o `mae` entre as duas curvas. Essas métricas devem ser do tipo numérico numpy. 

**Observe que essa função não recebe nenhum dado de tempo (coordenada x), pois assume que ambas as séries terão os mesmos valores para a coordenada x.**

In [ ]:
def compute_metrics(true_series, forecast):
    
    ### INICIE SEU CÓDIGO AQUI
    mse = None
    mae = None
    ### TERMINE SEU CÓDIGO AQUI

    return mse, mae

In [ ]:
# Teste sua função

# Defina uma série de dados fictícios para teste
zeros = np.zeros(5)
ones = np.ones(5)

mse, mae = compute_metrics(zeros, ones)
print(f"mse: {mse}, mae: {mae} para séries de zeros e previsão de uns\n")

mse, mae = compute_metrics(ones, ones)
print(f"mse: {mse}, mae: {mae} para séries de uns e previsão de uns\n")

print(f"metrics are numpy numeric types: {np.issubdtype(type(mse), np.number)}")

**Saída Esperada:**

```
mse: 1.0, mae: 1.0 para séries de zeros e previsão de uns

mse: 0.0, mae: 0.0 para séries de uns e previsão de uns

as métricas são tipos numéricos numpy: True
```

# Previsão

Agora que você tem uma maneira de medir o desempenho de suas previsões, é hora de começar a fazer algumas previsões.

Vamos começar de forma simples, usando uma previsão ingênua.

## Previsão ingênua

Defina a variável `naive_forecast` abaixo. Essa série deve ser idêntica à de validação, mas atrasada em um passo de tempo. Ela também recebe a etapa de tempo dividida da série para facilitar o cálculo da série atrasada.

**Observe que essa série deve deixar de fora o último elemento, pois ele não existe no conjunto de validação e você não conseguirá calcular as métricas de avaliação se esse elemento for mantido.**

Dica:

- Use toda a `SERIES` (treinamento e validação) e o `SPLIT_TIME` para calcular essa série.

In [ ]:
### INICIE O CÓDIGO AQUI
naive_forecast = None
### TERMINE O CÓDIGO AQUI

print(f" A série de validação tem forma {series_valid.shape}\n")
print(f" A previsão ingênua tem forma  {naive_forecast.shape}\n")
print(f" é comparável com a série de validação: {series_valid.shape == naive_forecast.shape}")

plt.figure(figsize=(10, 6))
plot_series(time_valid, series_valid, label="conjunto de validação")
plot_series(time_valid, naive_forecast, label="previsão ingênua")

**Saída Esperada:**

```
A série de validação tem forma (361,)

A previsão ingênua tem forma  (361,)

é comparable with validation series: True
```
<div>
<img src="images/naive.png" width="500"/>
</div>

Vamos dar um zoom no final do período de validação:

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, series_valid, start=330, end=361, label="conjunto de validação")
plot_series(time_valid, naive_forecast, start=330, end=361, label="previsão ingênua")

**Saída Esperada**

<div>
<img src="images/naive_zoom.png" width="500"/>
</div>

**Você deve ver que a previsão ingênua está 1 passo atrás da série temporal e que ambas as séries terminam na mesma etapa temporal.**

Agora vamos calcular o erro quadrático médio e o erro absoluto médio entre as previsões e os prognósticos no período de validação:

In [ ]:
mse, mae = compute_metrics(series_valid, naive_forecast)

print(f"mse: {mse:.2f}, mae: {mae:.2f} for naive forecast")

**Saída Esperada:**

```
mse: 19.58, mae: 2.60 for naive forecast
```

Essa é a nossa linha de base(_baseline_), agora vamos tentar uma média móvel.

## Média móvel

Complete a função `moving_average_forecast` abaixo. Essa função recebe uma "série" e um "tamanho_de_janela" e calcula a previsão de média móvel para cada ponto após os valores iniciais de "tamanho_de_janela".

**Essa função receberá a série completa e a série retornada será dividida para corresponder ao período de validação, de modo que sua função não precise levar em conta a correspondência da série com o período de validação.**

In [ ]:
def moving_average_forecast(series, window_size):
    """Prevê a média dos últimos valores.
        Se window_size=1, isso é equivalente à previsão ingênua"""
    
    forecast = []
    
    ### INICIE SEU CÓDIGO AQUI  
    for time in range(len(series) - window_size):
        forecast.append(series[time:time + window_size].mean())
        
    # Converta para vetor numpy
    np_forecast = None
    
    ### TERMINE SEU CÓDIGO AQUI  
    
    return np_forecast

Você não pode calcular a média móvel para os primeiros valores de `window_size`, pois não há valores suficientes para calcular a média desejada. Portanto, se você usar toda a `SERIES` e um `window_size` de 30, sua função deverá retornar uma série com o número de elementos igual a:

```python
len(SERIES) - 30
````

In [ ]:
print(f"A SERIES completa tem {len(SERIES)} portanto, a previsão da média móvel deve ter {len(SERIES)-30} elementos")

In [ ]:
# Teste sua função
moving_avg = moving_average_forecast(SERIES, window_size=30)
print(f"a previsão de média móvel com toda a SERIES tem forma: {moving_avg.shape}\n")

# Corte-o de modo que corresponda ao período de validação
moving_avg = moving_avg[1100 - 30:]
print(f"previsão de média móvel após o fatiamento ter forma: {moving_avg.shape}\n")
print(f"comparável com a série de validação: {series_valid.shape == moving_avg.shape}")


plt.figure(figsize=(10, 6))
plot_series(time_valid, series_valid)
plot_series(time_valid, moving_avg)

**Saída Esperada:**

```
a previsão de média móvel com toda a SERIES tem forma: (1431,)

previsão de média móvel após o fatiamento ter forma: (361,)

comparável com a série de validação: True
```
<div>
<img src="images/moving_avg.png" width="500"/>
</div>

In [ ]:
# Calcular métricas de avaliação
mse, mae = compute_metrics(series_valid, moving_avg)

print(f"mse: {mse:.2f}, mae: {mae:.2f} for moving average forecast")

**Saída Esperada:**

```
mse: 65.79, mae: 4.30 for moving average forecast
```

Isso é pior do que uma previsão ingênua! A média móvel não prevê tendência ou sazonalidade, portanto, vamos tentar removê-las usando a diferenciação. 

## Diferenciação

Como o período de sazonalidade é de 365 dias, subtrairemos o valor no momento *t* - 365 do valor no momento *t*.

Para isso, defina as variáveis `diff_series` e `diff_time` abaixo. Observe que `diff_time` é o valor da coordenada x de `diff_series`.

In [ ]:
### START CODE HERE
diff_series = None
diff_time = None
### END CODE HERE

print(f"A SERIES completa tem {len(SERIES)} portanto, a diferenciação deve ter {len(SERIES)-365} elementos\n")
print(f"A série diff tem forma: {diff_series.shape}\n")
print(f"A coordenada x da série de diferenças tem a forma: {diff_time.shape}\n")

plt.figure(figsize=(10, 6))
plot_series(diff_time, diff_series)
plt.show()

**Saída Esperada:**
```
A SERIES completa tem 1461 portanto, a diferenciação deve ter 1096 elementos

A série diff tem forma: (1096,)

A coordenada x da série de diferenças tem a forma: (1096,)
```
<div>
<img src="images/diff.png" width="500"/>
</div>

Ótimo, a tendência e a sazonalidade parecem ter desaparecido, então agora podemos usar a média móvel.

Defina a variável `diff_moving_avg`. 

**Observe que o `window_size` já foi definido e que você precisará executar o fatiamento correto para que a série corresponda ao período de validação.***

In [ ]:
### INICIE O CÓDIGO AQUI

# Aplicar a média móvel à série de diferenças
diff_moving_avg = moving_average_forecast(None, 50)

print(f"a previsão de média móvel com séries diferentes tem forma: {diff_moving_avg.shape}\n")

# Executar o fatiamento correto
diff_moving_avg = diff_moving_avg[None:]

### TERMINE O CÓDIGO AQUI

print(f"a previsão de média móvel com séries diferentes após o corte tem forma: {diff_moving_avg.shape}\n")
print(f"comparável com a série de validação: {series_valid.shape == diff_moving_avg.shape}")


plt.figure(figsize=(10, 6))
plot_series(time_valid, diff_series[1100 - 365:])
plot_series(time_valid, diff_moving_avg)
plt.show()

**Saída Esperada:**
```
a previsão de média móvel com séries diferentes tem forma: (1046,)

a previsão de média móvel com séries diferentes após o corte tem forma: (361,)

comparável com a série de validação: True
```
<div>
<img src="images/diff_moving.png" width="500"/>
</div>

Agora, vamos trazer de volta a tendência e a sazonalidade adicionando os valores anteriores de t - 365:

In [ ]:
### INICIE O CÓDIGO AQUI

# Corte a SERIES inteira para obter os valores anteriores
past_series = SERIES[None:None]

print(f"a série anterior tem forma: {past_series.shape}\n")


# Adicione o passado à média móvel da série de diferenças
diff_moving_avg_plus_past = past_series + None

### TERMINE O CÓDIGO AQUI

print(f"previsão de média móvel com séries diferentes mais o passado tem forma: {diff_moving_avg_plus_past.shape}\n")
print(f"comparável com a série de validação: {series_valid.shape == diff_moving_avg_plus_past.shape}")

plt.figure(figsize=(10, 6))
plot_series(time_valid, series_valid)
plot_series(time_valid, diff_moving_avg_plus_past)
plt.show()

**Saída Esperada:**

```
a série anterior tem forma: (361,)

previsão de média móvel com séries diferentes mais o passado tem forma: (361,)

ccomparável com a série de validação: True
```

<div>
<img src="images/plus_past.png" width="500"/>
</div>

In [ ]:
# Calcular métricas de avaliação
mse, mae = compute_metrics(series_valid, diff_moving_avg_plus_past)

print(f"mse: {mse:.2f}, mae: {mae:.2f} para média móvel mais previsão anterior")

**Expected Output:**

```
mse: 8.50, mae: 2.33 for moving average plus past forecast
```

Melhor do que a previsão ingênua, ótimo.

No entanto, as previsões parecem um pouco aleatórias demais porque estamos apenas adicionando valores passados, que eram ruidosos.

Você usará uma **média móvel centrada** com um tamanho de janela de `10` nos valores passados para remover parte do ruído. 

Se tiver dúvidas, você pode rever o laboratório [Previsão estatística em dados sintéticos](https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_time_series/class_01/TS%20-%20W1%20-%2008%20-%20Previs%C3%A3o.ipynb).

<br>
<details>
<summary><font size="2" color="darkgreen"><b>Clique aqui para dicas</b></font></summary>
    <ul>
        <li>Quando fatiando <code>os dados de SERIES</code>, comece com o <code>SPLIT_TIME</code> menos 370 passos
        <li>o tamanho  de <code>smooth_past_series</code> deve corresponder ao comprimento de <code>diff_moving_average</code>. Se você receber um erro de operando, é provável que isso não esteja sendo cumprido. Tente modificar o índice *end* do <code>SERIES</code> ao dividi-los para <code>smooth_past_series</code>.
        <li>Se estiver usando o laboratório "Previsão estatística em dados sintéticos" como referência, observe que ele usa um tamanho de janela de `11`. Este trabalho requer um tamanho de janela de `10`, portanto, você precisará modificar ligeiramente o código de acordo.
    </ul>
</details>



In [ ]:
### INICIE SEU CÓDIGO AQUI

# Executar a divisão correta de SERIES
smooth_past_series = moving_average_forecast(SERIES[None:None], 10)

print(f"a série passada suavizada tem forma: {smooth_past_series.shape}\n")

# Adicione os valores passados suavizados à média móvel da série de diferenças
diff_moving_avg_plus_smooth_past = smooth_past_series + None

### TERMINE SEU CÓDIGO AQUI

print(f"previsão de média móvel com séries diferentes mais o passado tem forma: {diff_moving_avg_plus_smooth_past.shape}\n")
print(f"comparável com a série de validação: {series_valid.shape == diff_moving_avg_plus_smooth_past.shape}")

plt.figure(figsize=(10, 6))
plot_series(time_valid, series_valid)
plot_series(time_valid, diff_moving_avg_plus_smooth_past)
plt.show()

**Saída Esperada:**

```
a série passada suavizada tem forma: (361,)

previsão de média móvel com séries diferentes mais o passado tem forma: (361,)

comparável com a série de validação: True
```

<div>
<img src="images/plus_smooth.png" width="500"/>
</div>

In [ ]:
# Calcular métricas de avaliação
mse, mae = compute_metrics(series_valid, diff_moving_avg_plus_smooth_past)

print(f"mse: {mse:.2f}, mae: {mae:.2f} para média móvel mais previsão passada suavizada")

**Saída Esperada:**

```
mse: 12.53, mae: 2.20 para média móvel mais previsão passada suavizada
```

**Parabéns por ter concluído a tarefa desta semana!**

Você implementou com sucesso funções para divisão e avaliação de séries temporais e, ao mesmo tempo, aprendeu a lidar com dados de séries temporais e a codificar métodos de previsão!

**Continue assim!**